# Assignment 2
### Olga Iarygina

### Calssification with the nearest neighbors

In this assignment I make a classification with the nearest neighbor method. To select the appropriate model I then proceed to cross-validation and standart data normalization for the aim of preprocessing.

In [1]:
import numpy as np

# read in the data
dataTrain = np.loadtxt('OccupancyTrain.csv', delimiter = ',')
dataTest = np.loadtxt('OccupancyTest.csv', delimiter = ',')
# split input variables and labels
XTrain = dataTrain[:, :-1] 
YTrain = dataTrain[:, -1] 
XTest = dataTest[:, :-1] 
YTest = dataTest[:, -1]

### ex1 :: nearst neighbor classification

First of all I just implement the knn-classifier with sklearn, specifying the parameter K as 1.
Actually we see, that our model is overfitted and predict the labels too well. We should also try different parameters for the model, since at this stage it is not clear whether this number of K is a good choice or not.

In [2]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(XTrain, YTrain)

KNeighborsClassifier(n_neighbors=1)

In [4]:
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(YTrain, knn.predict(XTrain))
acc_test = accuracy_score(YTest, knn.predict(XTest))
print("Accuracy on the train set:", acc_train, 
      "\nAccuracy on the test set:", acc_test)

Accuracy on the train set: 1.0 
Accuracy on the test set: 0.9775


Then I decided to try to implemet kNN classifier by my own.
I have searched for some solutions on StackOverflow and was inspired to make a class to make it work similar with a sklearn implementation.

Well, to build the algorithm let's first remember how does it works. First of all, we should calculate the Euclidean distance, which calculates the square root of the sum of the squared differences between two vectors.

Then we proceed to getting nearest neighbors.

The principle behind is that for a piece of data the neighbours are the k closest instances, where closure is determined by our Euclidean distance. Euclidean distance is just a length of the line between two points in space.

After the calculation the distances between each record in the dataset to a new piece of data, we sort all the records in the training dataset according to their distance to the new data, and select the k most similar neighbors.

After we just predict based on the fitted model.

In [5]:
class knn:

    def __init__(self, k = 5):  # I chose this default number as 1 neighbor was returned in sklearn by default
        self.k = k

    def euclidean_distance(self, a, b): # euclidean distance calculation
        dist = 0.0 
        
        for i in range(len(a)):
            dist += (a[i] - b[i]) ** 2
            euclidian_distance = np.sqrt(dist)
            
        return euclidian_distance


    def fit(self, XTrain, YTrain): # fitting the model
        self.XTrain = XTrain
        self.YTrain = YTrain

    def predict(self, XTest): # prediction based on getting the nearest neighbors
        pred = []

        for i in range(len(XTest)):  # here we iterate over the test set
            distances = []
            for point in self.XTrain:  # for each point in the train set we find a distance to each point in the test set
                point_dist = self.euclidean_distance(point, XTest[i])
                distances.append(point_dist)

            final = np.array(distances).argsort()[: self.k] # here I sort them and remain only specified number of k
            neighbors_list = {} # and then we count labels in YTrain
            
            for m in final:
                if self.YTrain[m] in neighbors_list:
                    neighbors_list[self.YTrain[m]] += 1
                else:
                    neighbors_list[self.YTrain[m]] = 1

            pred.append(max(neighbors_list, key = neighbors_list.get))
            
        return pred

In [6]:
my_knn = knn(k = 1)
my_knn.fit(XTrain, YTrain)

acc_train = accuracy_score(YTrain, my_knn.predict(XTrain))
acc_test = accuracy_score(YTest, my_knn.predict(XTest))
print("Accuracy on the train set:", acc_train, 
      "\nAccuracy on the test set:", acc_test)

Accuracy on the train set: 1.0 
Accuracy on the test set: 0.9775


Finally, it returns me the same accuracies as sklearn (after a thousand of tryings, to be honest). Now we can proceed to the following tasks.

My own implementation works much slower than KNeighborClassifier, therefore, I decided to use sklearn version further. Nevertheless, at different stages I checked the work of my model, and the accuracies were the same.

### ex2 :: cross-validation

Since the accuracy of the kNN model is determined by the k parameter selection, we need to somehow asses which value of k gives us the best results. To perform that I implement the cross-validation method. The principle behind is that the algorithm splits the dataset into a given number of folds N, and then each of the folds is used once to validate the models. The remaining folds are correspondingly N-1 left.

In this task I compared the performance of the model with 1, 3, 5, 7, 9 and 11 neighbors. I estimate the results with a 5-fold CV. After the camparison of different k parameter, I select the model with the lowest average loss and use it for further exercises.

In [7]:
from sklearn.model_selection import KFold

cv = KFold(n_splits = 5, random_state = 42, shuffle = True) # initiate 5-fold splitting

In [9]:
num_neighbors = [1, 2, 3, 5, 7, 9, 11] # identify differet values of k parameter to put them into for loop
num_accuracy = [] # here I will store the accuracies

for i in num_neighbors:
    accTest = []
    
    for train, test in cv.split(XTrain):
        XTrainCV, XTestCV, YTrainCV, YTestCV = XTrain[train], XTrain[test], YTrain[train], YTrain[test] # CV splitting
        
        knn = KNeighborsClassifier(n_neighbors = i) #fit the model
        knn.fit(XTrainCV, YTrainCV)
        accuracy = accuracy_score(YTestCV, knn.predict(XTestCV)) # get accuracy
        accTest.append(accuracy)
        
    num_accTest = np.mean(accTest) # find the mean between folds
    num_accuracy.append(num_accTest) # store the results
    
    print("Given k =", i, "the mean accuracy =", np.mean(accTest))

Given k = 1 the mean accuracy = 0.9800000000000001
Given k = 2 the mean accuracy = 0.9783333333333333
Given k = 3 the mean accuracy = 0.9916666666666666
Given k = 5 the mean accuracy = 0.9916666666666666
Given k = 7 the mean accuracy = 0.9916666666666666
Given k = 9 the mean accuracy = 0.9916666666666666
Given k = 11 the mean accuracy = 0.99


In [10]:
print("The best accuracy =", np.max(num_accuracy))
print("The best value of K: ", num_neighbors[np.argmax(num_accuracy)])

The best accuracy = 0.9916666666666666
The best value of K:  3


Overall, we can see, that the best model is when the number of K = 3. Noticeable that we have the same accuracies for the models with a higher number of K. But there is no need to add extra number of neighbors, since we already have a good result with K = 3. 

### ex3 :: evaluation of classification performance

In this task I simply evaluate the model with the number of K I revealed is the best by applying cross-validation.

Actually, we can see that even though the sample is splitted tinto test and train sets, the model seems to be overfitted.

In [11]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(XTrain, YTrain)

acc_train = accuracy_score(YTrain, knn.predict(XTrain))
acc_test = accuracy_score(YTest, knn.predict(XTest))
print("Accuracy on the train set:", acc_train, 
      "\nAccuracy on the test set:", acc_test)

Accuracy on the train set: 0.9933333333333333 
Accuracy on the test set: 0.9875


### ex4 :: data normalization

In this task I normalize the data. TWe use the standart normalization technique here, which is to generate zero-mean, unit variance input data. Firtly, I use scikit-learn implementation.

The appropriate solution from the suggeted in the text of the assignment is the first one. We cannot use the second one, because we are basing ourselves on the train data. The third one is inappropriate, because we are not supposed to use future data in normalization. They should be independent, otherwise testing will not make much sence.

In [12]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(XTrain) 
XTrainN = scaler.transform(XTrain) 
XTestN = scaler.transform(XTest)

In [13]:
num_neighbors = [1, 2, 3, 5, 7, 9, 11] # identify differet values of k parameter to put them into for loop
num_accuracy = [] # here I will store the accuracies

for i in num_neighbors:
    accTest = []
    
    for train, test in cv.split(XTrain):
        XTrainCV, XTestCV, YTrainCV, YTestCV = XTrainN[train], XTrainN[test], YTrain[train], YTrain[test] # CV splitting
        
        knn = KNeighborsClassifier(n_neighbors = i) #fit the model
        knn.fit(XTrainCV, YTrainCV)
        accuracy = accuracy_score(YTestCV, knn.predict(XTestCV)) # get accuracy
        accTest.append(accuracy)
        
    num_accTest = np.mean(accTest) # find the mean between folds
    num_accuracy.append(num_accTest) # store the results
    
    print("Given k =", i, "the mean accuracy =", np.mean(accTest))

Given k = 1 the mean accuracy = 0.9833333333333332
Given k = 2 the mean accuracy = 0.9850000000000001
Given k = 3 the mean accuracy = 0.9916666666666666
Given k = 5 the mean accuracy = 0.99
Given k = 7 the mean accuracy = 0.99
Given k = 9 the mean accuracy = 0.99
Given k = 11 the mean accuracy = 0.99


In [14]:
print("The best accuracy =", np.max(num_accuracy))
print("The best value of K: ", num_neighbors[np.argmax(num_accuracy)])

The best accuracy = 0.9916666666666666
The best value of K:  3


Considering the comparisons of different K, we have slightly different results in accuracies. But still the best model is still with the number of K equals 3. And the accuracy of this model on the train and test sets remained the same. I can suggest, that probably our data was already ditributed normally.

In [15]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(XTrainN, YTrain)

acc_train = accuracy_score(YTrain, knn.predict(XTrainN))
acc_test = accuracy_score(YTest, knn.predict(XTestN))

print("Accuracy on the train set:", acc_train, 
      "\nAccuracy on the test set:", acc_test)

Accuracy on the train set: 0.9933333333333333 
Accuracy on the test set: 0.9875


Then I decidet to try to perform normalization by my on, following the procedure described in the text of the assignment. So, we are first finding variance, mean and standard deviation, And them from each point we subtract the mean score, and divide this all by the standard deviation to get the centered results.

In [16]:
# train statistic values
XTrain_mean = np.mean(XTrain, axis = 0)
XTrain_std = np.std(XTrain, axis = 0)
XTrain_var = np.var(XTrain, axis = 0)

# test statistic values
XTest_mean = np.mean(XTest, axis = 0)
XTest_std = np.std(XTest, axis = 0)
XTest_var = np.var(XTest, axis = 0)

# normalization
XTrainN = (XTrain - XTrain_mean)/XTrain_std
XTestN = (XTest - XTrain_mean)/XTrain_std

And then I just test whether I got the same results with my own normalization.
Luckily, everything is fine.

In [17]:
num_neighbors = [1, 2, 3, 5, 7, 9, 11] # identify differet values of k parameter to put them into for loop
num_accuracy = [] # here I will store the accuracies

for i in num_neighbors:
    accTest = []
    
    for train, test in cv.split(XTrain):
        XTrainCV, XTestCV, YTrainCV, YTestCV = XTrainN[train], XTrainN[test], YTrain[train], YTrain[test] # CV splitting
        
        knn = KNeighborsClassifier(n_neighbors = i) #fit the model
        knn.fit(XTrainCV, YTrainCV)
        accuracy = accuracy_score(YTestCV, knn.predict(XTestCV)) # get accuracy
        accTest.append(accuracy)
        
    num_accTest = np.mean(accTest) # find the mean between folds
    num_accuracy.append(num_accTest) # store the results
    
    print("Given k =", i, "the mean accuracy =", np.mean(accTest))

Given k = 1 the mean accuracy = 0.9833333333333332
Given k = 2 the mean accuracy = 0.9850000000000001
Given k = 3 the mean accuracy = 0.9916666666666666
Given k = 5 the mean accuracy = 0.99
Given k = 7 the mean accuracy = 0.99
Given k = 9 the mean accuracy = 0.99
Given k = 11 the mean accuracy = 0.99


In [18]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(XTrainN, YTrain)

acc_train = accuracy_score(YTrain, knn.predict(XTrainN))
acc_test = accuracy_score(YTest, knn.predict(XTestN))

print("Accuracy on the train set:", acc_train, 
      "\nAccuracy on the test set:", acc_test)

Accuracy on the train set: 0.9933333333333333 
Accuracy on the test set: 0.9875
